In [1]:
import setpath
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from prepare.eegdataset import C_GeneralEEGImageDataset, DC_GeneralEEGImageDataset, GeneralEEGImageDataset, MySubset, N_GeneralEEGImageDataset
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
data_path = '/data0/tianjunchao/dataset/CVPR2021-02785/data/img_pkl/32x32'
# dataset = GeneralEEGImageDataset(path=data_path, n_channels=1, grid_size=8)
train_transforms = transforms.Compose([
    # transforms.RandomHorizontalFlip(),
    # transforms.RandomVerticalFlip(),
    # transforms.RandomRotation((30)),
    # transforms.RandomCrop(224),
    # transforms.ToTensor(),
    # one channel
    transforms.Resize((256, 256)),
    # transforms.RandomRotation(20),
    transforms.RandomCrop(224),
    transforms.Normalize([0.512, 0.512, 0.512], [0.228, 0.228, 0.228]),
])
n_samples = 1
dataset = C_GeneralEEGImageDataset(path=data_path, n_channels=1, grid_size=8, n_samples=n_samples)
# dataset = GeneralEEGImageDataset(path=data_path, n_channels=1, grid_size=8)
# dataset = GeneralEEGImageDataset(path=data_path, n_channels=1, grid_size=4, n_samples=n_samples)
scale = 2
dataset = MySubset(dataset, range(len(dataset)), transform=None)
# dataset = MySubset(dataset, range(len(dataset)), train_transforms)
dataloader = DataLoader(dataset, batch_size=128)

In [3]:
from prepare.data import filter_two_samples


images_list = []
label_list = []

# 创建一个字典来保存每个标签出现的次数
label_counts = {}

# 创建一个空列表来保存结果
result = []

# 设置目标标签出现的次数
TARGET_COUNT = 8

for i, (images, labels) in enumerate(dataloader):

    # 2 x batchsize x 3 x 224 x 224 to batchsize x 2 x 3 x 224 x 224
    images = torch.stack(images, dim=1)

    # image1, image2, labels = filter_two_samples(raw_image1, raw_image2, raw_labels,alpha=0)
    
    label = labels.numpy()
    for j in range(len(label)):
        
        if label_counts.get(label[j], 0) >= TARGET_COUNT:
            continue
        # 如果标签还没有出现过两次，将元素添加到结果列表中
        result.append((images[j], label[j]))
        label_counts[label[j]] = label_counts.get(label[j], 0) + 1
    # 如果每个类别的样本都出现了目标次数，跳出循环
    if all(count == TARGET_COUNT for count in label_counts.values()):
        break
    print('epoch',i,'len',len(labels),'label_counts',len(label_counts))
# 按照 y 从小到大排序
result.sort(key=lambda x: x[1])

epoch 0 len 128 label_counts 40
epoch 1 len 128 label_counts 40
epoch 2 len 128 label_counts 40


In [4]:
# 40个类别，每个类取出8x2个样本，均值和标准差
for i in range(40):

    target_y = i
    filtered_result = [xs for xs,y in result if y == target_y]
    x_list = []  
    for fr in filtered_result:
        x_list.extend(fr)
    print('target_y: ', target_y)
    mean_list = []
    std_list = []
    median_list = []
    for x in x_list:
        print('mean: ', torch.mean(x), 'median: ', torch.median(x), 'std: ', torch.std(x))
    print('-------------------------------------')

target_y:  0
mean:  tensor(0.3896) median:  tensor(0.3154) std:  tensor(0.2180)
mean:  tensor(0.4878) median:  tensor(0.4990) std:  tensor(0.1953)
mean:  tensor(0.4994) median:  tensor(0.5084) std:  tensor(0.1996)
mean:  tensor(0.5054) median:  tensor(0.5091) std:  tensor(0.1998)
mean:  tensor(0.5207) median:  tensor(0.5236) std:  tensor(0.1960)
mean:  tensor(0.5182) median:  tensor(0.5202) std:  tensor(0.1877)
mean:  tensor(0.5340) median:  tensor(0.5412) std:  tensor(0.2274)
mean:  tensor(0.5081) median:  tensor(0.5124) std:  tensor(0.2042)
mean:  tensor(0.5513) median:  tensor(0.5688) std:  tensor(0.1885)
mean:  tensor(0.5069) median:  tensor(0.5116) std:  tensor(0.2035)
mean:  tensor(0.5469) median:  tensor(0.5615) std:  tensor(0.2110)
mean:  tensor(0.5235) median:  tensor(0.5245) std:  tensor(0.1956)
mean:  tensor(0.4548) median:  tensor(0.4381) std:  tensor(0.2098)
mean:  tensor(0.5210) median:  tensor(0.5256) std:  tensor(0.2069)
mean:  tensor(0.4941) median:  tensor(0.4841) std

KeyboardInterrupt: 

In [5]:
# 将结果分成10段
sample_per_img = 16
num_classes = 40
num_images = TARGET_COUNT * num_classes * n_samples * scale // sample_per_img
# resutl = [(imgs1, label), (imgs2, label), ...]
segment_size = len(result) // num_images
segments = [result[i:i+segment_size] for i in range(0, len(result), segment_size)]

In [6]:
# def a function to show images
import os

from utils.eegutils import getNow


def show_images(it, imgs, labels,img_dir=None):
    # 将通道维度放在最后，以符合imshow的要求
    imgs = np.transpose(imgs, (0, 2, 3, 1))

    n_imgs = len(labels)
    ncols = 4
    nrows = n_imgs//ncols

    # print('imgs.shape: ', imgs.shape)
    # 打印16张图像
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10, 10))
    # 显示每个图像的标签


    for i, ax in enumerate(axes.flat):
        ax.set_title(labels[i])
        ax.imshow(imgs[i])
        ax.set_axis_off()

    # 调整子图之间的间距
    plt.tight_layout()

    if img_dir is not None:
    # 显示图像
    # plt.show()
        plt.savefig(img_dir+ str(it)+' - ' +str(labels[0])+'~'+str(labels[len(labels)-1])+'.png')
        plt.close()

def show_channels(img):
    plt.imshow(img)

    # 显示图像
    plt.show()
    # 分别画出每个通道的图像
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(9, 3))

    for i, ax in enumerate(axes):
        ax.imshow(img[..., i])
        ax.set_title('Channel {}'.format(i+1))

    # 调整子图之间的间距
    plt.tight_layout()

    # 显示图像
    plt.show()



# draw images in each segment
def draw_list(segment):
    images = []
    labels = []
    # c,w,h = segment[0][0].shape
    _, c,w,h = segment[0][0].shape
    # 遍历segment中的每个元素
    for i, data in enumerate(segment):
        xs, y = data
        for x in xs:
            images.append(x)
            labels.append(y)
            
    images = torch.stack(images, dim=0).view(len(labels), c, w, h)
    return images,labels

In [7]:
img_dir = '4.materials/labels/'+getNow()+'/'
if not os.path.exists(img_dir):
    os.makedirs(img_dir)
for i, segment in enumerate(segments):
    images,labels = draw_list(segment)
    show_images(i,images, labels,img_dir=img_dir)

In [ ]:
segment[0][0].shape